<a href="https://colab.research.google.com/github/frankwillard/NBA-Hall-Of-Fame-Model/blob/main/Hall_of_Fame_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Packages and Data

In [27]:
# Load packages

from math import exp
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, get_scorer, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE, SelectFromModel, SequentialFeatureSelector

In [28]:
# Read in cleaned player data
model_df = pd.read_csv("https://raw.githubusercontent.com/frankwillard/NBA-Hall-Of-Fame-Model/main/Scraped%20Player%20Data.csv", index_col=0)
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4977 entries, 0 to 4976
Data columns (total 67 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          4977 non-null   object 
 1   Eligible        4977 non-null   int64  
 2   Position        4977 non-null   object 
 3   Hall_of_Fame    4977 non-null   int64  
 4   MVP             4977 non-null   int64  
 5   Finals_MVP      4977 non-null   int64  
 6   NBA_Champ       4977 non-null   int64  
 7   All_NBA         4977 non-null   int64  
 8   All_Defensive   4977 non-null   int64  
 9   Def_POY         4977 non-null   int64  
 10  All_Star        4977 non-null   int64  
 11  Scoring_Champ   4977 non-null   int64  
 12  TRB_Champ       4977 non-null   int64  
 13  AST_Champ       4977 non-null   int64  
 14  STL_Champ       4977 non-null   int64  
 15  BLK_Champ       4977 non-null   int64  
 16  All_ABA         4977 non-null   int64  
 17  ABA_Champ       4977 non-null   i

### Data Cleaning

In [29]:
# Replace all instances of -999 with NA (consider doing this in scraper to eliminate a step)
model_df = model_df.replace(-999, np.nan)

In [30]:
# Reduce number of possible positions to guard, forward, center
model_df.loc[model_df['Position'] == 'Center/Forward', 'Position'] = 'Center'
model_df.loc[model_df['Position'].isin(['PointGuard', 'ShootingGuard', 'Guard/Forward']), 'Position'] = 'Guard'
model_df.loc[model_df['Position'].isin(['SmallForward', 'PowerForward', 'Forward/Guard', 'Forward/Center']), 'Position'] = 'Forward'

In [31]:
# Combine individual ABA and NBA accolades
model_df['All_League'] = model_df['All_NBA'] + model_df['All_ABA']
model_df['Champ'] = model_df['NBA_Champ'] + model_df['ABA_Champ']

In [32]:
hofers = model_df['Hall_of_Fame'].value_counts()[1]

In [33]:
# Count number of NAs by column:
for col in model_df.columns:
  na_df = model_df[model_df[col].isna()]
  if len(na_df) > 0:
    try:
      na_rows = na_df['Hall_of_Fame'].value_counts()[1]
    except KeyError:
      na_rows = 0
    print(f"{col}:\t{len(model_df[model_df[col].isna()])} nulls \t{na_rows}/{hofers} HOFers are null")

3P_per_game:	1118 nulls 	53/140 HOFers are null
3PA_per_game:	1118 nulls 	53/140 HOFers are null
2P_per_game:	1118 nulls 	53/140 HOFers are null
2PA_per_game:	1118 nulls 	53/140 HOFers are null
ORB_per_game:	949 nulls 	39/140 HOFers are null
DRB_per_game:	949 nulls 	39/140 HOFers are null
TRB_per_game:	288 nulls 	1/140 HOFers are null
STL_per_game:	1180 nulls 	41/140 HOFers are null
BLK_per_game:	1180 nulls 	41/140 HOFers are null
GS_totals:	1689 nulls 	66/140 HOFers are null
FG%_totals:	34 nulls 	0/140 HOFers are null
3P_totals:	1118 nulls 	53/140 HOFers are null
3PA_totals:	1118 nulls 	53/140 HOFers are null
3P%_totals:	1627 nulls 	54/140 HOFers are null
2P_totals:	1118 nulls 	53/140 HOFers are null
2PA_totals:	1118 nulls 	53/140 HOFers are null
2P%_totals:	1162 nulls 	53/140 HOFers are null
eFG%_totals:	1146 nulls 	53/140 HOFers are null
FT%_totals:	241 nulls 	0/140 HOFers are null
ORB_totals:	949 nulls 	39/140 HOFers are null
DRB_totals:	949 nulls 	39/140 HOFers are null
TRB_totals

In [34]:
# Columns with -999s
for col in model_df.columns:
  if len(model_df[model_df[col] == -999]) > 0:
    print(col, "-", len(model_df[model_df[col] == -999]))

#### **What to do with NA values**
<br/>

Columns to drop:
`GS_totals`, `Trp_Dbl_totals`, `ORB_per_game`, `DRB_per_game`, `ORB_totals`, `DRB_totals`, `3P%_totals`, `2P%_totals`, `eFG%_totals`, `OWS_advanced`, `DWS_advanced`, `WS/48_advanced`, `OBPM_advanced`, `DBPM_advanced`
<br/>
<br/>

Columns to consider dropping: `3P_per_game`, `3PA_per_game`, `3P_totals`, `3PA_totals`, `FG%_totals` (these players never took a shot), `FT%_totals` (these players never took a FT)
<br/>
<br/>

Columns to impute from FGM, FGA, etc.:
`2P_per_game`, `2PA_per_game`, `2P_totals`, `2PA_totals`
<br/>
<br/>

Columns to fill with league average:
`PER_advanced`, `VORP_advanced` (consider some more advanced PER/VORP)
<br/>
<br/>

Columns to make 0:
`WS_advanced`, `BPM_advanced` (consider some more advanced imputation for BPM)
<br/>
<br/>

Columns to make 0 or fill with mean (undecided):,
        "`TS%_advanced` (these players never took a shot or free throw)
<br/>
<br/>

Columns to fill with mean by position:,
`PTS_per_game`, `TRB_per_game`, `AST_per_game`, `STL_per_game`, `BLK_per_game`, `TRB_totals`, `AST_totals`, `STL_totals`, `BLK_totals`

In [35]:
# Fill NAs accordingly
def fillNulls(model_df):
  cols_to_zero = ['WS_advanced', 'BPM_advanced', '3P_per_game', '3PA_per_game', '3P_totals', '3PA_totals', 'FG%_totals', 'FT%_totals', 'TS%_advanced']
  model_df[cols_to_zero] = model_df[cols_to_zero].fillna(0) # fill cols with 0
  
  cols_to_avg = ['PER_advanced', 'VORP_advanced', '3P%_totals', '2P%_totals', 'eFG%_totals']
  model_df[cols_to_avg] = model_df[cols_to_avg].fillna(model_df[cols_to_avg].mean()) # fill cols with avg
  
  cols_to_position_avg = ['TRB_per_game', 'AST_per_game', 'STL_per_game', 'BLK_per_game', 'TRB_totals', 'AST_totals', 'STL_totals', 'BLK_totals']
  model_df[cols_to_position_avg] = model_df.groupby("Position")[cols_to_position_avg].transform(lambda x: x.fillna(x.mean())) # fills cols with avg by position
  
  cols_to_fill = ['2P_per_game', '2PA_per_game', '2P_totals', '2PA_totals']
  cols_to_fill_with = ['FG_per_game', 'FGA_per_game', 'FG_totals', 'FGA_totals']
  model_df[cols_to_fill] = model_df[cols_to_fill].fillna(model_df[cols_to_fill_with]) # fill 2P shooting columns with FG columns

  return model_df
fillNulls(model_df)

,Player,Eligible,Position,Hall_of_Fame,MVP,Finals_MVP,NBA_Champ,All_NBA,All_Defensive,Def_POY,...,OWS_advanced,DWS_advanced,WS_advanced,WS/48_advanced,OBPM_advanced,DBPM_advanced,BPM_advanced,VORP_advanced,All_League,Champ
0,Alaa Abdelnaby,1,Forward,0,0,0,0,0,0,0,...,0.7,4.1,4.8,0.072,-2.9,-0.9,-3.8,-1.500000,0,0
1,Zaid Abdul-Aziz,1,Center,0,0,0,0,0,0,0,...,5.9,11.6,17.5,0.076,0.6,-0.2,0.4,2.700000,0,0
2,Kareem Abdul-Jabbar,1,Center,1,6,2,6,15,11,0,...,178.9,94.5,273.4,0.228,4.1,1.6,5.7,85.700000,15,6
3,Mahmoud Abdul-Rauf,1,Guard,0,0,0,0,0,0,0,...,16.7,8.4,25.2,0.077,0.7,-1.5,-0.8,4.500000,0,0
4,Tariq Abdul-Wahad,1,Guard,0,0,0,0,0,0,0,...,-0.6,4.1,3.5,0.035,-2.6,-0.4,-3.0,-1.200000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972,Jim Zoet,1,Center,0,0,0,0,0,0,0,...,-0.1,0.0,-0.1,-0.123,-5.6,0.2,-5.4,-0.100000,0,0
4973,Bill Zopf,1,Guard,0,0,0,0,0,0,0,...,-0.5,0.4,-0.1,-0.011,NaN,NaN,0.0,3.434634,0,0
4974,Ivica Zubac,0,Center,0,0,0,0,0,0,0,...,16.4,9.6,26.1,0.183,0.3,0.4,0.6,4.500000,0,0
4975,Matt Zunic,1,Guard,0,0,0,0,0,0,0,...,0.2,1.8,2.0,NaN,NaN,NaN,0.0,3.434634,0,0


In [36]:
# Columns to exclude

#NBA and ABA were combined for All League, Championship
#Games played not a stat
#Name, eligibility, position not relevant
#Attempts does not mean anything- can just pass in makes and percentage
#FT, FG encoded in points

exclude_cols = ['Player', 'Eligible', 'Position', 'NBA_Champ', 'All_NBA', 'All_ABA', 'ABA_Champ', 'G_totals', 
                '3PA_totals', 'FTA_totals', 'FGA_totals', 
                'FTA_per_game', 'FGA_per_game', '3PA_per_game',
                'FT_per_game', 'FT_totals', 'FG_per_game', 'FG_totals']

# Count number of NAs by column:
for col in model_df.columns:
  na_df = model_df[model_df[col].isna()]
  if len(na_df) > 0:
    try:
      na_rows = na_df['Hall_of_Fame'].value_counts()[1]
    except KeyError:
      na_rows = 0
    print(f"{col}:\t{len(model_df[model_df[col].isna()])} nulls \t{na_rows}/{hofers} HOFers are null")
    exclude_cols.append(col)

2P_per_game:	1118 nulls 	53/140 HOFers are null
2PA_per_game:	1118 nulls 	53/140 HOFers are null
ORB_per_game:	949 nulls 	39/140 HOFers are null
DRB_per_game:	949 nulls 	39/140 HOFers are null
GS_totals:	1689 nulls 	66/140 HOFers are null
2P_totals:	1118 nulls 	53/140 HOFers are null
2PA_totals:	1118 nulls 	53/140 HOFers are null
ORB_totals:	949 nulls 	39/140 HOFers are null
DRB_totals:	949 nulls 	39/140 HOFers are null
Trp_Dbl_totals:	4526 nulls 	53/140 HOFers are null
OWS_advanced:	1 nulls 	0/140 HOFers are null
DWS_advanced:	1 nulls 	0/140 HOFers are null
WS/48_advanced:	344 nulls 	1/140 HOFers are null
OBPM_advanced:	1185 nulls 	41/140 HOFers are null
DBPM_advanced:	1185 nulls 	41/140 HOFers are null


In [37]:
exclude_cols

['Player',
 'Eligible',
 'Position',
 'NBA_Champ',
 'All_NBA',
 'All_ABA',
 'ABA_Champ',
 'G_totals',
 '3PA_totals',
 'FTA_totals',
 'FGA_totals',
 'FTA_per_game',
 'FGA_per_game',
 '3PA_per_game',
 'FT_per_game',
 'FT_totals',
 'FG_per_game',
 'FG_totals',
 '2P_per_game',
 '2PA_per_game',
 'ORB_per_game',
 'DRB_per_game',
 'GS_totals',
 '2P_totals',
 '2PA_totals',
 'ORB_totals',
 'DRB_totals',
 'Trp_Dbl_totals',
 'OWS_advanced',
 'DWS_advanced',
 'WS/48_advanced',
 'OBPM_advanced',
 'DBPM_advanced']

### Feature Selection

In [38]:
# All remaining columns
model_df.columns

Index(['Player', 'Eligible', 'Position', 'Hall_of_Fame', 'MVP', 'Finals_MVP',
       'NBA_Champ', 'All_NBA', 'All_Defensive', 'Def_POY', 'All_Star',
       'Scoring_Champ', 'TRB_Champ', 'AST_Champ', 'STL_Champ', 'BLK_Champ',
       'All_ABA', 'ABA_Champ', 'ROY', 'FG_per_game', 'FGA_per_game',
       '3P_per_game', '3PA_per_game', '2P_per_game', '2PA_per_game',
       'FT_per_game', 'FTA_per_game', 'ORB_per_game', 'DRB_per_game',
       'TRB_per_game', 'AST_per_game', 'STL_per_game', 'BLK_per_game',
       'PTS_per_game', 'G_totals', 'GS_totals', 'FG_totals', 'FGA_totals',
       'FG%_totals', '3P_totals', '3PA_totals', '3P%_totals', '2P_totals',
       '2PA_totals', '2P%_totals', 'eFG%_totals', 'FT_totals', 'FTA_totals',
       'FT%_totals', 'ORB_totals', 'DRB_totals', 'TRB_totals', 'AST_totals',
       'STL_totals', 'BLK_totals', 'PTS_totals', 'Trp_Dbl_totals',
       'PER_advanced', 'TS%_advanced', 'OWS_advanced', 'DWS_advanced',
       'WS_advanced', 'WS/48_advanced', 'OBPM_advanced

In [39]:
# Split into two dataframes: one where the players are eligible for the Hall of Fame, one where they are not

all_cols_eligible_df = (model_df[model_df['Eligible'] == 1]).loc[:, ~model_df.columns.isin(exclude_cols)]
#all_cols_noneligible_df = (model_df[model_df['Eligible'] == 0]).loc[:, ~model_df.columns.isin(exclude_cols)]

In [40]:
all_cols_eligible_df

,Hall_of_Fame,MVP,Finals_MVP,All_Defensive,Def_POY,All_Star,Scoring_Champ,TRB_Champ,AST_Champ,STL_Champ,...,STL_totals,BLK_totals,PTS_totals,PER_advanced,TS%_advanced,WS_advanced,BPM_advanced,VORP_advanced,All_League,Champ
0,0,0,0,0,0,0,0,0,0,0,...,71.000000,69.000000,1465,13.000000,0.532,4.8,-3.8,-1.500000,0,0
1,0,0,0,0,0,0,0,0,0,0,...,131.000000,205.000000,4557,15.100000,0.479,17.5,0.4,2.700000,0,0
2,1,6,2,11,0,19,2,1,0,0,...,1160.000000,3189.000000,38387,24.600000,0.592,273.4,5.7,85.700000,15,6
3,0,0,0,0,0,0,0,0,0,0,...,487.000000,46.000000,8553,15.400000,0.506,25.2,-0.8,4.500000,0,0
4,0,0,0,0,0,0,0,0,0,0,...,184.000000,83.000000,1830,11.400000,0.467,3.5,-3.0,-1.200000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,0,0,0,0,0,0,0,0,0,0,...,35.000000,31.000000,458,6.100000,0.474,0.0,-5.2,-1.400000,0,0
4972,0,0,0,0,0,0,0,0,0,0,...,1.000000,3.000000,2,-0.800000,0.200,-0.1,-5.4,-0.100000,0,0
4973,0,0,0,0,0,0,0,0,0,0,...,270.596835,56.631013,118,9.600000,0.391,-0.1,0.0,3.434634,0,0
4975,0,0,0,0,0,0,0,0,0,0,...,270.596835,56.631013,273,11.212562,0.368,2.0,0.0,3.434634,0,0


In [41]:
# Scale features and split eligible dataset into dependent and independent variables

sc0 = StandardScaler()

X_all = pd.DataFrame(sc0.fit_transform(all_cols_eligible_df.iloc[:,1:]), index=all_cols_eligible_df.index, columns=all_cols_eligible_df.columns[1:])
y_all = all_cols_eligible_df.iloc[:,0]

In [42]:
X_all.columns

Index(['MVP', 'Finals_MVP', 'All_Defensive', 'Def_POY', 'All_Star',
       'Scoring_Champ', 'TRB_Champ', 'AST_Champ', 'STL_Champ', 'BLK_Champ',
       'ROY', '3P_per_game', 'TRB_per_game', 'AST_per_game', 'STL_per_game',
       'BLK_per_game', 'PTS_per_game', 'FG%_totals', '3P_totals', '3P%_totals',
       '2P%_totals', 'eFG%_totals', 'FT%_totals', 'TRB_totals', 'AST_totals',
       'STL_totals', 'BLK_totals', 'PTS_totals', 'PER_advanced',
       'TS%_advanced', 'WS_advanced', 'BPM_advanced', 'VORP_advanced',
       'All_League', 'Champ'],
      dtype='object')

#### Experimenting with Various Feature Selectors

In [43]:
#Select top k features based on mutual info regression
kbest_selector = SelectKBest(mutual_info_regression, k = 10)
kbest_selector.fit(X_all, y_all)
kbest_cols = list(X_all.columns[kbest_selector.get_support()])

kbest_cols

['All_Star',
 'PTS_per_game',
 'TRB_totals',
 'AST_totals',
 'PTS_totals',
 'PER_advanced',
 'WS_advanced',
 'BPM_advanced',
 'VORP_advanced',
 'All_League']

In [44]:
rfe_selector = RFE(estimator=LogisticRegression(max_iter=120),n_features_to_select = 10, step = 1)
rfe_selector.fit(X_all, y_all)
rfe_cols = list(X_all.columns[rfe_selector.get_support()])

rfe_cols

['MVP',
 'All_Star',
 'Scoring_Champ',
 'TRB_per_game',
 'BLK_per_game',
 'FG%_totals',
 'TRB_totals',
 'PER_advanced',
 'WS_advanced',
 'Champ']

In [45]:
sfm_selector = SelectFromModel(estimator=LogisticRegression())
sfm_selector.fit(X_all, y_all)
sfm_cols = list(X_all.columns[sfm_selector.get_support()])

sfm_cols

['MVP',
 'All_Star',
 'Scoring_Champ',
 'TRB_per_game',
 'BLK_per_game',
 'PTS_per_game',
 'FG%_totals',
 'FT%_totals',
 'TRB_totals',
 'BLK_totals',
 'PER_advanced',
 'WS_advanced',
 'Champ']

In [46]:
list(set(sfm_cols) & set(rfe_cols) & set(kbest_cols))

['WS_advanced', 'All_Star', 'PER_advanced', 'TRB_totals']

The SFS technique takes awhile

In [47]:
#sfs_selector = SequentialFeatureSelector(estimator=LogisticRegression(), n_features_to_select = 10, cv =5, direction ='backward')
#sfs_selector.fit(X_all, y_all)
#X_all.columns[sfs_selector.get_support()]

#### Selecting Our Columns

In [50]:
# 'Finals_MVP', 'BLK_totals', 'VORP_advanced', 'STL_totals', 'TS%_advanced', 'All_Defensive', 'AST_Champ', 'TRB_totals', 'AST_totals'

# 'BPM_advanced', 'WS_advanced',
model_cols = ['Player', 'Eligible', 'Hall_of_Fame', 'Def_POY', 'All_Star', 'All_League', 'Champ', # accolades
       'PTS_per_game', 'WS_advanced', 'TRB_totals', 'PER_advanced'] # stats

eligible_df = (model_df[model_df['Eligible'] == 1]).loc[:, model_df.columns.isin(model_cols)]
noneligible_df = (model_df[model_df['Eligible'] == 0]).loc[:, model_df.columns.isin(model_cols)]

In [51]:
# Dropping players that are unpredictable (reasoning explained below)
extraneous_players = ['Maurice Stokes', 'Bill Bradley', 'Toni Kukoč',
       'Calvin Murphy', 'Vlade Divac', 'Buddy Jeannette',
       'Dražen Petrović', 'Al Cervi', 'Arvydas Sabonis',
       'Šarūnas Marčiulionis', 'Dino Radja', 'Chuck Cooper',
       'Bob Houbregs']

eligible_df = eligible_df[~eligible_df['Player'].isin(extraneous_players)]

### Train/Test Split

In [52]:
# Split training set into X and y
X_eligible = eligible_df.iloc[:, 3:].values
y_eligible = eligible_df.iloc[:, 2].values

In [53]:
# Train-test split dividing HOF eligible players into a training set and a validation set
X_training, X_validation, y_train, y_val = train_test_split(eligible_df, y_eligible, test_size = 0.25, random_state = 0)

In [54]:
X_train = X_training.iloc[:,3:].values
X_val = X_validation.iloc[:,3:].values

In [55]:
X_test = noneligible_df.iloc[:, 3:]
y_test = noneligible_df.iloc[:, 2]

### Feature Scaling

In [56]:
# Scale features
sc1 = StandardScaler()
X_eligible = sc1.fit_transform(X_eligible)

# Scale whole matrix of features to prevent information leakage
# Scale for training set and validation set
sc2 = StandardScaler()
X_train = sc2.fit_transform(X_train)
X_val = sc2.transform(X_val)

### Model Selection

In [57]:
# Define classifiers / regressors

CLASSIFIERS = [
    LogisticRegression(),
    XGBClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    GaussianNB(),
    KNeighborsClassifier(),
    AdaBoostClassifier(),
    SVC(kernel = 'rbf')
  ]

In [58]:
# Define accuracy metrics

METRICS = [
    'f1',
    'accuracy',
    'precision',
    'recall',
    'average_precision'
]

Accuracy- Percentage of samples correctly classified

Precision- Of players model identified as HOF players, how many were truly HOF

Recall- Of true HOF players, how many did model identify as HOF

F1- Harmonic mean of precision and recall

In [59]:
# Function that fits, predicts, and evaluates on different model types
def get_metrics(classifier, X, y_true):
  print(classifier)
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X)
  #clf_val_preds = classifier.predict(X_val)

  output_metrics = []

  for metric in METRICS:
    score = get_scorer(metric)._score_func(y_true, y_pred)
    output_metrics.append(score)

  return output_metrics


In [60]:
training_metrics = []
val_metrics = []

for classifier in CLASSIFIERS:
  clf_train_metrics = get_metrics(classifier, X_train, y_train)
  val_train_metrics = get_metrics(classifier, X_val, y_val)
  
  training_metrics.append([classifier] + clf_train_metrics)
  val_metrics.append([classifier] + val_train_metrics)

LogisticRegression()
LogisticRegression()
XGBClassifier()
XGBClassifier()
DecisionTreeClassifier()
DecisionTreeClassifier()
RandomForestClassifier()
RandomForestClassifier()
SVC()
SVC()
GaussianNB()
GaussianNB()
KNeighborsClassifier()
KNeighborsClassifier()
AdaBoostClassifier()
AdaBoostClassifier()
SVC()
SVC()


In [61]:
train_metrics_df = pd.DataFrame(data = training_metrics, columns=['Classifier'] + METRICS)
val_metrics_df = pd.DataFrame(data = val_metrics, columns=['Classifier'] + METRICS)

In [62]:
# Output modeling metrics
train_metrics_df.sort_values(by='f1', ascending=False)

,Classifier,f1,accuracy,precision,recall,average_precision
2,DecisionTreeClassifier(),1.000000,1.000000,1.000000,1.000000,1.000000
3,"(DecisionTreeClassifier(max_features='auto', r...",1.000000,1.000000,1.000000,1.000000,1.000000
7,"(DecisionTreeClassifier(max_depth=1, random_st...",1.000000,1.000000,1.000000,1.000000,1.000000
1,XGBClassifier(),0.979167,0.998693,0.979167,0.979167,0.959421
4,SVC(),0.926316,0.995425,0.936170,0.916667,0.860770
8,SVC(),0.926316,0.995425,0.936170,0.916667,0.860770
6,KNeighborsClassifier(),0.897297,0.993791,0.932584,0.864583,0.810545
0,LogisticRegression(),0.893617,0.993464,0.913043,0.875000,0.802835
5,GaussianNB(),0.648649,0.966013,0.480000,1.000000,0.480000


In [63]:
val_metrics_df.sort_values(by='f1', ascending=False)

,Classifier,f1,accuracy,precision,recall,average_precision
7,"(DecisionTreeClassifier(max_depth=1, random_st...",0.862069,0.992157,0.925926,0.806452,0.752597
1,XGBClassifier(),0.814815,0.990196,0.956522,0.709677,0.687645
6,KNeighborsClassifier(),0.754717,0.987255,0.909091,0.645161,0.597295
0,LogisticRegression(),0.745098,0.987255,0.950000,0.612903,0.594023
2,DecisionTreeClassifier(),0.727273,0.985294,0.833333,0.645161,0.548419
4,SVC(),0.727273,0.985294,0.833333,0.645161,0.548419
8,SVC(),0.727273,0.985294,0.833333,0.645161,0.548419
3,"(DecisionTreeClassifier(max_features='auto', r...",0.720000,0.986275,0.947368,0.580645,0.562830
5,GaussianNB(),0.696629,0.973529,0.534483,1.000000,0.534483


### Hyperparameter Optimization

In [64]:
logistic_classifier = LogisticRegression(random_state= 0)

In [65]:
#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)

In [66]:
# Define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=logistic_classifier, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1',error_score=0)
grid_result = grid_search.fit(X_eligible, y_eligible)

In [67]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print()
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.883661 using {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}

0.880795 (0.073478) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.880795 (0.073478) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.880795 (0.073478) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.879466 (0.072331) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.879466 (0.072331) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.883661 (0.066451) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.881715 (0.068288) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.881715 (0.068288) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.881156 (0.066707) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.868535 (0.074621) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.868535 (0.074621) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.866720 (0.067900) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.813012 (0.076462) wit

### Modeling

In [68]:
#from sklearn.preprocessing import PolynomialFeatures
#poly_reg = PolynomialFeatures(degree = 2)
#X_poly = poly_reg.fit_transform(X_train)

In [69]:
# Fit a classifier with parameters found above
classifier = LogisticRegression(random_state = 1, **grid_result.best_params_)
classifier.fit(X_train, y_train)

LogisticRegression(C=10, random_state=1, solver='liblinear')

### Predictions

In [70]:
# Predict both class and probability for the training set 
y_train_pred_probs = classifier.predict_proba(X_train)[:, 1]
y_train_pred = classifier.predict(X_train)

In [71]:
# Predict both class and probability for the test set 
y_val_pred_probs = classifier.predict_proba(X_val)[:, 1]
y_val_pred = classifier.predict(X_val)

In [72]:
#eligible_df['pred'] = y_train_pred_probs
X_training['pred'] = y_train_pred_probs
X_validation['pred'] = y_val_pred_probs

#### Borderline Correct Positive HOF Predictions

In [73]:
#eligible_df[(eligible_df['pred'] > 0.5) & (eligible_df['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=True)
X_training[(X_training['pred'] > 0.5) & (X_training['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=True)[:5]

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
4492,Wes Unseld,1,1,0,5,10.8,13769.0,16.0,110.1,1,1,0.504837
2910,Dick McGuire,1,1,0,7,8.0,2784.0,15.6,50.9,1,0,0.557051
4937,George Yardley,1,1,0,6,19.2,4220.0,20.6,58.5,2,0,0.560948
2673,Clyde Lovellette,1,1,0,4,17.0,6663.0,21.7,70.6,1,3,0.583699
3136,Chris Mullin,1,1,0,5,18.2,4034.0,18.8,93.1,4,0,0.599421


In [74]:
X_validation[(X_validation['pred'] > 0.5) & (X_validation['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=True)[:5]

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
4606,Bobby Wanzer,1,1,0,5,12.2,1979.0,17.3,63.9,3,1,0.504006
1583,Gail Goodrich,1,1,0,5,18.6,3279.0,16.7,76.3,1,1,0.524446
4663,Chris Webber,1,1,0,5,20.7,8124.0,20.9,84.7,5,0,0.646861
1848,Connie Hawkins,1,1,0,5,18.7,5450.0,19.9,76.7,3,1,0.653624
1702,Richie Guerin,1,1,0,6,17.3,4278.0,17.3,69.2,3,0,0.669731


#### Under predictions

##### Expected Under Predictions:

<br/>

#### **Old players:**

<br/>


Bob Houbregs- A few seasons with mediocre stats, great college player but no NBA accolades and no other league. No clue why he is in. From 50s

<br/>

Chuck Cooper- 6.7 PPG and no accolades. Instrumental to NBA as broke the color barrier

<br/>

Al Cervi- 4 years of recorded stats with Syracuse (49-53). Dominated NBL with scoring champion, championship, 4 all league

<br/>

Buddy Jeannette	- From 40s, 3 seasons in BAA, but played in late 30s, early 40s with no stats for NBL- seeral first team/titles

<br/>
<br/>


#### **European players:**

<br/>

Dino Radja - Croatian player, only 4 years in NBA (with very good stats) but made it for Europe play

<br/>

Šarūnas Marčiulionis - 8 solid years in NBA, 8 years before in USSR league, made it for foreign/olympic play

<br/>

Arvydas Sabonis	- Many great years in Lithuania/Spain unaccounted for, solid 7 years in NBA in 30s


<br/>

Dražen Petrović	- 5 solid years in NBA, played in Yugoslavia/Spain


<br/>

Vlade Divac	- Many years in NBA with solid stats, no accolades. Did well in Yugoslav league, won Europa POY in 7 years before NBA

<br/>

Toni Kukoc- Very solid NBA career and picked up some championships. Played in Italy/Yugoslavia before (9 years). Won Europa POY, Euroscar POY, many Euroleage Championships

<br/>
<br/>


#### **Other:**

Bill Bradley- Not great individual stats, exceptional college player and 2 time champ but not above average. 70s so not that old

<br/>

Calvin Murphy- Very good pro career with not many accolades. Three time all american in college

<br/>

Maurice Stokes - From 50s, 3 years in NBA putting up 17-17 type numbers. Injured and paralyzed after three years with 3 all NBA. Good college player too

<br/>



##### **Legitimately wrong:**

Guy Rodgers- Long NBA career, 4x all star, 2x ast champ

Ralph Sampson- 6 solid years in NBA (of his 9), 4x all star, legend in college

Bill Walton - MVP, Legend in college, 2x all star, 2x all league. Can be fixed by adding MVP variable but may hurt with other players

Walt Bellamy- 4x all star, tons of points (43) and rebounds (12) over career

Wes Unseld- Solid accolades, consistent 15-15 guy earlier in career

Jamaal Wilkes- Solid contributor to many championships, also won All D which is currently not on here

Players to look into:

 'Walt Bellamy',
       'Maurice Cheeks', 'Earl Monroe', 'Arnie Risen', 'Tom Gola',
       'Frank Ramsey', 'Wes Unseld', 'Andy Phillip', 'Bob Dandridge',
       'David Thompson'

In [75]:
#eligible_df[(eligible_df['pred'] < 0.5) & (eligible_df['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=False)
X_training[(X_training['pred'] < 0.5) & (X_training['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
4754,Jamaal Wilkes,1,1,0,3,17.7,5117.0,16.5,71.3,0,4,0.493721
2759,Pete Maravich,1,1,0,5,24.2,2747.0,18.4,46.7,4,0,0.481977
489,Carl Braun,1,1,0,5,13.5,2122.0,15.8,64.3,2,1,0.458260
1561,Manu Ginóbili,1,1,0,2,13.3,3697.0,20.2,106.4,2,4,0.451019
3497,Andy Phillip,1,1,0,5,9.1,2395.0,14.1,60.5,2,1,0.368075
3068,Earl Monroe,1,1,0,4,18.8,2796.0,17.2,77.4,1,1,0.265850
308,Walt Bellamy,1,1,0,4,20.1,14241.0,19.9,130.0,0,0,0.207465
761,Maurice Cheeks,1,1,0,4,11.1,3088.0,16.5,103.5,0,1,0.204959
1452,Joe Fulks,1,1,0,2,16.4,1379.0,10.9,29.2,4,1,0.033013
4601,Bill Walton,1,1,0,2,13.3,4923.0,20.0,39.3,2,2,0.032582


In [76]:
X_validation[(X_validation['pred'] < 0.5) & (X_validation['Hall_of_Fame'] == 1)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
2211,Gus Johnson,1,1,0,5,16.2,7624.0,16.7,36.9,4,1,0.492942
3944,Charlie Scott,1,1,0,5,20.7,2846.0,15.8,41.4,2,1,0.469939
967,Bob Dandridge,1,1,0,4,18.5,5715.0,16.7,80.3,1,2,0.467957
4376,David Thompson,1,1,0,5,22.7,2446.0,19.9,63.3,3,0,0.454437
989,Bob Davies,1,1,0,4,14.3,980.0,18.1,49.7,5,1,0.346174
2399,Bernard King,1,1,0,4,22.5,5060.0,19.2,75.4,4,0,0.305334
1576,Tom Gola,1,1,0,5,11.3,5617.0,14.2,53.2,1,1,0.286925
3730,Arnie Risen,1,1,0,4,12.0,5011.0,16.7,56.0,1,2,0.251121
2284,K.C. Jones,1,1,0,0,7.4,2399.0,10.4,38.6,0,8,0.243560
3627,Frank Ramsey,1,1,0,0,13.4,3410.0,15.6,49.2,0,7,0.182577


#### Over predictions

Larry Foust deserves to be in HOF. Also has 94% rating on Bball ref

Amare Stoudemire (72% bball ref), Chauncey (84% on bball ref), Shawn Marion (76% on bball ref) are newer to ballot

Larry Costello is in HOF as Contributor- this led to confusion as one of his key contributions was his play. 71% on bball ref

Tom Sanders in HOF as contributor but 15% rating on bball ref

Max Zaslofsky	

Legitimate wrong predictions:

Jimmy Jones- ABA Legend, very efficient. Think ABA was less respected when he was in it than Erving. Shorter career

Shawn Kemp (39% bball ref)- Lower longevity

Donnie Freeman- ABA Star i late 60s early 70s

Walter Davis (31% bball ref)

Jermaine O'Neal (32% bball ref)

In [77]:
#eligible_df[(eligible_df['pred'] > 0.5) & (eligible_df['Hall_of_Fame'] == 0)].sort_values(by='pred', ascending=False)
X_training[(X_training['pred'] > 0.5) & (X_training['Hall_of_Fame'] == 0)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
4240,Amar'e Stoudemire,1,0,0,6,18.9,6632.0,21.8,92.5,5,0,0.857446
351,Chauncey Billups,1,0,0,5,15.2,2992.0,18.8,120.8,3,1,0.794745
2281,Jimmy Jones,1,0,0,6,16.6,2930.0,17.1,78.8,3,0,0.707552
2363,Shawn Kemp,1,0,0,6,14.6,8834.0,19.1,89.5,3,0,0.689012
1440,Donnie Freeman,1,0,0,5,18.9,2292.0,18.2,58.0,4,1,0.663794
1028,Walter Davis,1,0,0,6,18.9,3053.0,19.1,76.9,2,0,0.658111
888,Larry Costello,1,0,0,6,12.2,2705.0,14.5,62.7,1,1,0.634997
3298,Jermaine O'Neal,1,0,0,6,13.2,7261.0,17.9,66.0,3,0,0.566189


In [78]:
X_validation[(X_validation['pred'] > 0.5) & (X_validation['Hall_of_Fame'] == 0)].sort_values(by='pred', ascending=False)

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
1412,Larry Foust,1,0,0,8,13.7,8041.0,19.8,79.2,2,0,0.924441


#### Checking for overpredictions of irrelevant players with great advanced stats

In [92]:
X_training.sort_values(by='PER_advanced', ascending=False)[:5]

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
1944,Steven Hill,1,0,0,0,2.0,3.0,88.3,0.0,0,0,0.000016
2646,Ian Lockhart,1,0,0,0,4.0,0.0,76.3,0.0,0,0,0.000023
3931,Dave Scholz,1,0,0,0,2.0,0.0,67.6,0.0,0,0,0.000024
1472,Chad Gallagher,1,0,0,0,3.0,0.0,66.8,0.1,0,0,0.000026
2609,Cedric Lewis,1,0,0,0,1.3,2.0,62.7,0.1,0,0,0.000025


In [91]:
X_validation.sort_values(by='PER_advanced', ascending=False)[:5]

,Player,Eligible,Hall_of_Fame,Def_POY,All_Star,PTS_per_game,TRB_totals,PER_advanced,WS_advanced,All_League,Champ,pred
4704,Tyson Wheeler,1,0,0,0,4.0,0.0,76.1,0.1,0,0,0.000023
1547,Trey Gilder,1,0,0,0,1.0,1.0,30.9,0.0,0,0,0.000047
2878,John McCullough,1,0,0,0,2.6,4.0,30.7,0.1,0,0,0.000053
3299,Shaquille O'Neal,1,1,0,15,23.7,13099.0,26.4,181.7,14,4,1.000000
1365,Richard Fisher,1,0,0,0,3.1,32.0,25.1,0.2,0,0,0.000062


Model does well at handling these extreme cases

### Model Coefficients

In [80]:
for col, coef in zip(model_cols[3:], classifier.coef_[0]):
  print(f"{col}: {exp(coef)}")

Def_POY: 1.1904295556966724
All_Star: 6.6259885865832455
All_League: 1.4086808906983885
Champ: 0.9581321511501842
PTS_per_game: 0.8769937823219144
WS_advanced: 1.7406724520277068
TRB_totals: 1.446413977667046
PER_advanced: 1.9342550783751138


Massive collinearity problem here causing points to hurt here

In [81]:
#for col, coef in zip(poly_reg.get_feature_names(), classifier.coef_[0]):
#  print(f"{col}: {exp(coef)}")

### Confusion Matrix

In [82]:
# Output confuson matrix for the training set
cm = confusion_matrix(y_train, y_train_pred)
#[00 01]
#[10 11]
print(cm)
accuracy_score(y_train, y_train_pred)

[[2956    8]
 [  11   85]]


0.9937908496732026

In [83]:
# Output confusion matrix for the validation set
cm = confusion_matrix(y_val, y_val_pred)
#[00 01]
#[10 11]
print(cm)
accuracy_score(y_val, y_val_pred)

[[988   1]
 [ 11  20]]


0.9882352941176471

### Other Metrics

In [66]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2964
           1       0.91      0.88      0.89        96

    accuracy                           0.99      3060
   macro avg       0.95      0.94      0.95      3060
weighted avg       0.99      0.99      0.99      3060



In [67]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       989
           1       0.95      0.65      0.77        31

    accuracy                           0.99      1020
   macro avg       0.97      0.82      0.88      1020
weighted avg       0.99      0.99      0.99      1020

